In [4]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import numba

# Define constants
frequency = 40000 # Hz
wavelength = 343 / frequency # m
amplitude = 400 # Increased amplitude
rho_air = 1.225 # kg/m^3
c_air = 343 # m/s
mu_air = 1.82e-5 # Pa*s
radius = 0.01 # m
rho_particle = 0.01 # kg/m^3
c_particle = 900 # m/s
mass = (4/3) * np.pi * radius**3 * rho_particle # kg

# Gorkov potential coefficients
K1 = (1 / (4 * np.pi * c_air**2 * rho_air)) * (1 - (rho_air / rho_particle))
K2 = (1 / (8 * np.pi * c_air**2 * rho_air)) * ((rho_particle - rho_air) / (2 * rho_particle + rho_air))

# Gorkov potential function with effective amplitude modification
@numba.njit
def gorkov_potential(z, transducer_distance):
    k = 2 * np.pi / wavelength
    amplitude_effective = amplitude * (1 + np.cos(k * (transducer_distance - 2 * z)))
    return (4 / 3) * np.pi * radius**3 * (K1 * amplitude_effective**2 * (np.cos(2 * k * z) - 1) + 3 * K2 * amplitude_effective**2 * np.cos(2 * k * z))

# Ball motion equation
@numba.njit
def ball_motion(state, dt, transducer_distance):
    z, v = state
    k = 2 * np.pi / wavelength
    d_gorkov_potential_dz = -(8 / 3) * np.pi * radius**3 * K1 * amplitude**2 * k * np.sin(2 * k * z) * (1 + np.cos(k * (transducer_distance - 2 * z)))
    acoustic_force_gradient = d_gorkov_potential_dz
    drag_force = -6 * np.pi * mu_air * radius * v
    gravity_force = -mass * 9.81
    total_force = acoustic_force_gradient + drag_force + gravity_force
    dv_dt = total_force / mass
    dz_dt = v
    return [z + dz_dt * dt, v + dv_dt * dt]

# Initial conditions and simulation
z0 = 50.5 * wavelength
v0 = 0 # m/s
state0 = [z0, v0]
transducer_distance = 100 * wavelength
t = np.linspace(0, 5, 1000000)
solution = odeint(5, state0, t, args=(transducer_distance,))

# Plot results
plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.plot(t, solution[:, 0])
plt.title('Position of the particle')
plt.xlabel('Time (s)')
plt.ylabel('Position (m)')

plt.subplot(2, 1, 2)
plt.plot(t, solution[:, 1])
plt.title('Velocity of the particle')
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')

plt.tight_layout()
plt.show()


error: The function and its Jacobian must be callable functions.